In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
_dataset_path = '/content/drive/MyDrive/Celerates/Notebook/Final Project/datasets/acceptance_dataset.csv'
_output_path = '/content/drive/MyDrive/Celerates/Notebook/Final Project/outputs'

### **Setting-Up Configuration**

In [4]:
import os
import torch
import random
import numpy as np

In [5]:
CSV_PATH = _dataset_path
TEXT_COL = "resume_clean"
LABEL_COL = "label"
MODEL_NAME = "distilbert-base-uncased"
MAX_LENGTH = 256
BATCH_SIZE = 8
VAL_BATCH_SIZE = 16
LR = 3e-5
WEIGHT_DECAY = 0.01
EPOCHS = 10
SEED = 42
OUTPUT_DIR = _output_path
GRADIENT_ACCUMULATION_STEPS = 1
MAX_GRAD_NORM = 1.0
WARMUP_RATIO = 0.1
FP16 = True
NUM_WORKERS = 2

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# reproducibility
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)
print("Device:", DEVICE)

Device: cuda


### **Setup Dataset**

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv(CSV_PATH)
df.head()

,Role,Resume,Decision,Reason_for_decision,Job_Description
0,E-commerce Specialist,Here's a professional resume for Jason Jones:\...,reject,Lacked leadership skills for a senior position.,Be part of a passionate team at the forefront ...
1,Game Developer,Here's a professional resume for Ann Marshall:...,select,Strong technical skills in AI and ML.,Help us build the next-generation products as ...
2,Human Resources Specialist,Here's a professional resume for Patrick Mccla...,reject,Insufficient system design expertise for senio...,We need a Human Resources Specialist to enhanc...
3,E-commerce Specialist,Here's a professional resume for Patricia Gray...,select,Impressive leadership and communication abilit...,Be part of a passionate team at the forefront ...
4,E-commerce Specialist,Here's a professional resume for Amanda Gross:...,reject,Lacked leadership skills for a senior position.,We are looking for an experienced E-commerce S...


### **Getting Text and Label**

In [8]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
# Get text col

stop = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = text.lower()
    text = " ".join([w for w in text.split() if w not in stop])
    return text

df["resume_clean"] = df["Resume"].apply(clean_text)

df.head()

,Role,Resume,Decision,Reason_for_decision,Job_Description,resume_clean
0,E-commerce Specialist,Here's a professional resume for Jason Jones:\...,reject,Lacked leadership skills for a senior position.,Be part of a passionate team at the forefront ...,professional resume jason jones jason jones e ...
1,Game Developer,Here's a professional resume for Ann Marshall:...,select,Strong technical skills in AI and ML.,Help us build the next-generation products as ...,professional resume ann marshall ann marshall ...
2,Human Resources Specialist,Here's a professional resume for Patrick Mccla...,reject,Insufficient system design expertise for senio...,We need a Human Resources Specialist to enhanc...,professional resume patrick mcclain patrick mc...
3,E-commerce Specialist,Here's a professional resume for Patricia Gray...,select,Impressive leadership and communication abilit...,Be part of a passionate team at the forefront ...,professional resume patricia gray patricia gra...
4,E-commerce Specialist,Here's a professional resume for Amanda Gross:...,reject,Lacked leadership skills for a senior position.,We are looking for an experienced E-commerce S...,professional resume amanda gross amanda gross ...


In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
# Get label col

'''
label mapping:
    0: rejected
    1: selected (not accepted)
'''

le = LabelEncoder()
df['label'] = le.fit_transform(df['Decision'])
num_classes = df["label"].nunique()

df.head(10)

,Role,Resume,Decision,Reason_for_decision,Job_Description,resume_clean,label
0,E-commerce Specialist,Here's a professional resume for Jason Jones:\...,reject,Lacked leadership skills for a senior position.,Be part of a passionate team at the forefront ...,professional resume jason jones jason jones e ...,0
1,Game Developer,Here's a professional resume for Ann Marshall:...,select,Strong technical skills in AI and ML.,Help us build the next-generation products as ...,professional resume ann marshall ann marshall ...,1
2,Human Resources Specialist,Here's a professional resume for Patrick Mccla...,reject,Insufficient system design expertise for senio...,We need a Human Resources Specialist to enhanc...,professional resume patrick mcclain patrick mc...,0
3,E-commerce Specialist,Here's a professional resume for Patricia Gray...,select,Impressive leadership and communication abilit...,Be part of a passionate team at the forefront ...,professional resume patricia gray patricia gra...,1
4,E-commerce Specialist,Here's a professional resume for Amanda Gross:...,reject,Lacked leadership skills for a senior position.,We are looking for an experienced E-commerce S...,professional resume amanda gross amanda gross ...,0
5,Mobile App Developer,"Here's a sample resume for Jose Hall, a skille...",reject,No experience in back-end development.,We need a Mobile App Developer to enhance our ...,sample resume jose hall skilled mobile app dev...,0
6,UX Designer,Here's a professional resume for Rachael Newma...,reject,Insufficient system design expertise for senio...,Help us build the next-generation products as ...,professional resume rachael newman applying ro...,0
7,Cloud Engineer,"Here's a sample resume for Jessica Hall, a can...",reject,Needs improvement in machine learning algorithms.,We're seeking a talented Cloud Engineer to wor...,sample resume jessica hall candidate applying ...,0
8,Digital Marketing Specialist,Here's a sample resume for Jonathan Powers:\n\...,select,Impressive leadership and communication abilit...,"As a Digital Marketing Specialist, you'll lead...",sample resume jonathan powers jonathan powers ...,1
9,AI Researcher,Here's a sample resume for Zachary Ward:\n\nZa...,select,Excellent full-stack development experience.,If you're passionate about software engineerin...,sample resume zachary ward zachary ward ai res...,1


In [12]:
assert TEXT_COL in df.columns and LABEL_COL in df.columns, f"CSV must contain columns: {TEXT_COL}, {LABEL_COL}"
df = df[[TEXT_COL, LABEL_COL]].dropna().reset_index(drop=True)
df[LABEL_COL] = df[LABEL_COL].astype(int)

print("Dataset size:", len(df))
print(df[LABEL_COL].value_counts(normalize=True))

Dataset size: 10174
label
0    0.502654
1    0.497346
Name: proportion, dtype: float64


### **Train test splitting**

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train_df, val_df = train_test_split(
    df, test_size=0.2,
    stratify=df[LABEL_COL],
    random_state=SEED
    )

print("Train:", len(train_df), "Val:", len(val_df))

Train: 8139 Val: 2035


### **Tokenization**

In [15]:
from transformers import DistilBertTokenizerFast

In [16]:
tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

### **Apply dataset to model**

In [17]:
from torch.utils.data import Dataset, DataLoader
from typing import List

In [18]:
class SimpleTextDataset(Dataset):
    def __init__(self, texts: List[str], labels: List[int], tokenizer: DistilBertTokenizerFast, max_length: int):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])
        enc = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_attention_mask=True,
            return_tensors="pt",
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

def collate_fn(batch):
    input_ids = torch.stack([x["input_ids"] for x in batch])
    attention_mask = torch.stack([x["attention_mask"] for x in batch])
    labels = torch.stack([x["labels"] for x in batch])
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [19]:
# Data Loader
train_dataset = SimpleTextDataset(train_df[TEXT_COL].tolist(), train_df[LABEL_COL].tolist(), tokenizer, MAX_LENGTH)
val_dataset = SimpleTextDataset(val_df[TEXT_COL].tolist(), val_df[LABEL_COL].tolist(), tokenizer, MAX_LENGTH)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=NUM_WORKERS, pin_memory=True)

In [20]:
# quick sanity
batch = next(iter(train_loader))
print("Batch shapes:", batch["input_ids"].shape, batch["attention_mask"].shape, batch["labels"].shape)

Batch shapes: torch.Size([8, 256]) torch.Size([8, 256]) torch.Size([8])


### **Build Model**

In [21]:
from transformers import DistilBertModel
import torch.nn as nn

In [22]:
backbone = DistilBertModel.from_pretrained(MODEL_NAME)
backbone.to(DEVICE)
backbone.train()  # we'll call .train()/ .eval() during loop

hidden_size = backbone.config.hidden_size  # 768 usually

# classification head as nn.Sequential (no custom class)
classifier_head = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(hidden_size, 256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(0.15),
    nn.Linear(256, 2)   # logits for 2 classes
)
classifier_head.to(DEVICE)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Sequential(
  (0): Dropout(p=0.3, inplace=False)
  (1): Linear(in_features=768, out_features=256, bias=True)
  (2): ReLU()
  (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Dropout(p=0.15, inplace=False)
  (5): Linear(in_features=256, out_features=2, bias=True)
)

In [23]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import autocast, GradScaler

In [24]:
param_groups = [
    {"params": backbone.parameters()},
    {"params": classifier_head.parameters()},
]
optimizer = AdamW(param_groups, lr=LR, weight_decay=WEIGHT_DECAY)

# scheduler warmup linear
total_steps = max(1, (len(train_loader) // max(1, GRADIENT_ACCUMULATION_STEPS)) * EPOCHS)
warmup_steps = int(total_steps * WARMUP_RATIO)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

loss_fn = nn.CrossEntropyLoss()
scaler = GradScaler()

/tmp/ipython-input-3904055366.py:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


### **Train, Monitor, and Evaluate Functions**

In [25]:
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [26]:
def compute_metrics(probs, labels):
    preds = np.argmax(probs, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

def train_one_epoch(epoch, backbone, head, dataloader, optimizer, scheduler, device, scaler, accumulation_steps=1):
    backbone.train()
    head.train()
    running_loss = 0.0
    all_probs = []
    all_labels = []
    optimizer.zero_grad()
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Train {epoch}", leave=False)
    for step, batch in pbar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with autocast(enabled=(FP16 and device.startswith("cuda"))):
            outputs = backbone(input_ids=input_ids, attention_mask=attention_mask)
            cls_emb = outputs.last_hidden_state[:, 0, :]          # [CLS]
            logits = head(cls_emb)
            loss = loss_fn(logits, labels) / accumulation_steps

        scaler.scale(loss).backward()

        if (step + 1) % accumulation_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(list(backbone.parameters()) + list(head.parameters()), max_norm=MAX_GRAD_NORM)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            if scheduler is not None:
                scheduler.step()

        running_loss += loss.item() * accumulation_steps
        probs = torch.softmax(logits.detach().cpu(), dim=1).numpy()
        all_probs.extend(probs)
        all_labels.extend(labels.detach().cpu().numpy())
        pbar.set_postfix(loss=running_loss / (step + 1))

    avg_loss = running_loss / len(dataloader)
    metrics = compute_metrics(np.array(all_probs), np.array(all_labels))
    return avg_loss, metrics

def evaluate(backbone, head, dataloader, device):
    backbone.eval()
    head.eval()
    running_loss = 0.0
    all_probs = []
    all_labels = []
    with torch.no_grad():
        pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc="Eval", leave=False)
        for step, batch in pbar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = backbone(input_ids=input_ids, attention_mask=attention_mask)
            cls_emb = outputs.last_hidden_state[:, 0, :]
            logits = head(cls_emb)
            loss = loss_fn(logits, labels)
            running_loss += loss.item()

            probs = torch.softmax(logits.detach().cpu(), dim=1).numpy()
            all_probs.extend(probs)
            all_labels.extend(labels.detach().cpu().numpy())
            pbar.set_postfix(loss=running_loss / (step + 1))

    avg_loss = running_loss / len(dataloader)
    metrics = compute_metrics(np.array(all_probs), np.array(all_labels))
    return avg_loss, metrics

### **Run Training**

In [28]:
best_val_f1 = -1.0
training_stats = {"train_loss": [], "train_acc": [], "train_f1": [], "val_loss": [], "val_acc": [], "val_f1": []}

for epoch in range(1, EPOCHS + 1):
    train_loss, train_metrics = train_one_epoch(epoch, backbone, classifier_head, train_loader, optimizer, scheduler, DEVICE, scaler, accumulation_steps=GRADIENT_ACCUMULATION_STEPS)
    val_loss, val_metrics = evaluate(backbone, classifier_head, val_loader, DEVICE)

    print(f"Epoch {epoch} | Train loss {train_loss:.4f} Acc {train_metrics['accuracy']:.4f} F1 {train_metrics['f1']:.4f}")
    print(f"          Val   loss {val_loss:.4f} Acc {val_metrics['accuracy']:.4f} F1 {val_metrics['f1']:.4f}")

    training_stats["train_loss"].append(train_loss)
    training_stats["train_acc"].append(train_metrics["accuracy"])
    training_stats["train_f1"].append(train_metrics["f1"])
    training_stats["val_loss"].append(val_loss)
    training_stats["val_acc"].append(val_metrics["accuracy"])
    training_stats["val_f1"].append(val_metrics["f1"])

    # save best
    if val_metrics["f1"] > best_val_f1:
        best_val_f1 = val_metrics["f1"]

        # save backbone + head state_dicts and tokenizer
        torch.save(backbone.state_dict(), os.path.join(OUTPUT_DIR, "backbone.pkl"))
        torch.save(classifier_head.state_dict(), os.path.join(OUTPUT_DIR, "head.pkl"))
        tokenizer.save_pretrained(OUTPUT_DIR)
        print(f"Saved best model (f1={best_val_f1:.4f}) to {OUTPUT_DIR}")


Train 1:   0%|          | 0/1018 [00:00<?, ?it/s]

/tmp/ipython-input-2682018772.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(FP16 and device.startswith("cuda"))):


Eval:   0%|          | 0/128 [00:00<?, ?it/s]

Epoch 1 | Train loss 0.7077 Acc 0.5431 F1 0.5632
          Val   loss 0.6333 Acc 0.5661 F1 0.2672
Saved best model (f1=0.2672) to /content/drive/MyDrive/Celerates/Notebook/Final Project/outputs


Train 2:   0%|          | 0/1018 [00:00<?, ?it/s]

...

Epoch 9 | Train loss 0.4096 Acc 0.8549 F1 0.8533
          Val   loss 1.3563 Acc 0.5794 F1 0.5542


Train 10:   0%|          | 0/1018 [00:00<?, ?it/s]

/tmp/ipython-input-2682018772.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(FP16 and device.startswith("cuda"))):


Eval:   0%|          | 0/128 [00:00<?, ?it/s]

Epoch 10 | Train loss 0.3915 Acc 0.8647 F1 0.8615
          Val   loss 1.3441 Acc 0.5794 F1 0.5579
